In [1]:
import os
import csv
from typing import List, Tuple
from concurrent.futures import ProcessPoolExecutor

from ortools.algorithms.python import knapsack_solver

# Code:

In [2]:
def best_subset_knapsack(target: int, nums: List[int]) -> Tuple[int, List[int]]:
    """
    Finds the best subset of numbers that adds up to a target sum
    using Knapsack Solver.
    Args:
        target (int): The target sum.
        nums (List[int]): List of numbers to choose from.
    Returns:
        Tuple[int, List[int]]: The best sum found and the subset of numbers.
    """
    if target < 0 or not nums:
        return 0, []
    solver = knapsack_solver.KnapsackSolver(
        knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        "SubsetSum"
    )
    solver.init(nums, [nums], [target])
    best_value = solver.solve()
    chosen = []
    for i, v in enumerate(nums):
        if solver.best_solution_contains(i):
            chosen.append(v)
    return best_value, chosen


def parse_ints(row: List[str]) -> List[int]:
    """
    Parse a CSV row into a list of integers.
    Args:
        row (List[str]): A CSV row as a list of strings.
    Returns:
        List[int]: A list of integers parsed from the CSV row.
    """
    return [int(x) for x in row if x.strip() != ""]


def process_row(args: Tuple[int, List[int], int]) -> Tuple[int, int, int, List[int]]:
    """
    Procesa una fila: calcula el mejor subset sum.
    Args:
        args: (target, smalls, line_no)
    Returns:
        (line_no, target, best_sum, subset)
    """
    target, smalls, line_no = args
    best_sum, subset = best_subset_knapsack(target, smalls)
    return (line_no, target, best_sum, subset)


def main() -> None:
    """
    Main function to process the CSV file and find the best subset sum in parallel.
    """
    path: str = "data/sum_numbers/data.csv"
    rows_to_process = []
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        for line_no, row in enumerate(reader, start=1):
            if not row:
                continue
            vals = parse_ints(row)
            target = vals[0]
            smalls = vals[1:]
            rows_to_process.append((target, smalls, line_no))
        num_workers = os.cpu_count()
        with ProcessPoolExecutor(max_workers=num_workers) as executor:
            results = executor.map(process_row, rows_to_process)
            for line_no, target, best_sum, subset in results:
                print(f"Row {line_no}: target={target} best_sum={best_sum} subset={subset}")

In [3]:
main()

Row 1: target=854 best_sum=854 subset=[136, 133, 114, 89, 134, 92, 92, 64]
Row 2: target=277 best_sum=277 subset=[64, 113, 100]
Row 3: target=376 best_sum=376 subset=[110, 48, 57, 96, 65]
Row 4: target=493 best_sum=490 subset=[59, 122, 106, 145, 58]
Row 5: target=554 best_sum=554 subset=[142, 135, 49, 98, 130]
Row 6: target=727 best_sum=727 subset=[110, 83, 139, 149, 134, 62, 50]
Row 7: target=678 best_sum=678 subset=[35, 110, 145, 128, 87, 82, 91]
Row 8: target=239 best_sum=239 subset=[29, 137, 73]
Row 9: target=391 best_sum=391 subset=[120, 92, 65, 114]
Row 10: target=326 best_sum=324 subset=[108, 78, 138]
Row 11: target=817 best_sum=817 subset=[75, 97, 25, 112, 132, 147, 36, 145, 48]
Row 12: target=675 best_sum=673 subset=[52, 143, 126, 114, 152, 86]
Row 13: target=115 best_sum=115 subset=[25, 39, 51]
Row 14: target=552 best_sum=552 subset=[122, 40, 43, 155, 92, 100]
Row 15: target=483 best_sum=482 subset=[76, 87, 70, 99, 35, 115]
Row 16: target=728 best_sum=728 subset=[55, 132, 131

# Questions:

- Why you think your approach is the best for this problem?

Perfect Algorithm Match: This approach uses the right algorithm for the job. Instead of trying every possible combination (which takes forever with big datasets) or using memory-heavy dynamic programming, the knapsack method gives us the most straightforward and efficient way to solve subset sum problems.

Reliable and Tested Solution: The solution uses OR-Tools, which is a proven library that companies around the world have been using for years. This means we don't have to write our own complex algorithms that might have bugs or fail in unexpected ways, giving us confidence that it will work reliably in real applications.

Easy to Modify and Expand: The code is organized so that different parts handle different jobs - reading data, doing calculations, and running things in parallel. This makes it easy to change one part without breaking everything else. For example, we could switch to a different solver, change how we read files, or adjust how many processes we use, all without rewriting the whole program.

- What its advantages are?

The knapsack algorithm is the standard solution for subset sum problems and works well even with large numbers or big datasets. OR-Tools uses optimized C/C++ code wrapped for Python, which gives us fast performance while keeping python's ease of use. The algorithm is smart about skipping unnecessary calculations, making it much faster than trying every possible combination.

The parallel processing works perfectly here because each CSV row can be handled independently. Using ProcessPoolExecutor avoids Python's threading limitations by creating separate processes, which lets us use all CPU cores effectively. The program automatically detects how many cores are available and processes the CSV file without loading everything into memory at once, which prevents memory problems with large files.

- What limitations it might have?

The biggest limitation is that OR-Tools only works with whole numbers, so negative values or decimals won't work directly. For decimal numbers, you need to multiply everything by 10, 100, 1000, etc. to make them whole numbers, but this can cause precision errors and make calculations slower. Very large scaling factors might even cause the numbers to become too big for the system to handle.

Even with all the optimizations, this is still a hard computational problem that can take exponential time in worst cases. Memory usage can become a real issue with very large datasets or when the target numbers are huge. The program also depends on installing OR-Tools with its C++ components, which can be tricky in some environments.